# **EDUARDO PEREZ CHAVARRIA. FT17**
## Proyecto individual
### ETL 1. Archivo a analizar Steam_games

## **Cargamos librerías**

In [40]:
import pandas as pd
import numpy as np

import gzip
import json
import re

## **1. Importar La base de datos**

In [41]:
#señalamos donde está la base de datos.
ruta = "bases/steam_games.json.gz"

# creamos una lista para almacenar los diccionarios, ya que vamos a trabajar con json
lista_datos = []

#La base tiene formato gz, entonces descomprimimos primero con gzip y vamos cargando los datos en diccionario
with gzip.open(ruta, "rt", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line)
        lista_datos.append(data)

# convertimos la lista de datos en DF
df_juegos = pd.DataFrame(lista_datos)

# Visualizamos el df. No usamos print porque es más práctico 
df_juegos


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


## **2. Eliminar columnas innecesarias**

eliminamos las columnas que no se necesitan para el modelo ni para las consultas

In [42]:
df_juegos = df_juegos.drop(["title", "specs", "tags", "url", "reviews_url"], axis=1)
df_juegos.columns
print(df_juegos.shape)


(120445, 8)


## **3. Eliminando valores nulos en todo DF**

Necesitamos empezar a limpiar quitando valores nulos. dropear na puede hacerse con el argumento any por defecto o solo para las filas que son todas na y conservar la mayor cantidad de datos. Para tomar esta decisión observaremos primero el porcentaje de nas por columna


In [43]:
porcentaje_nulos = df_juegos.isna().mean() * 100
print(porcentaje_nulos)

publisher       80.004982
genres          76.045498
app_name        73.321433
release_date    75.035909
price           74.463033
early_access    73.319773
id              73.321433
developer       76.058782
dtype: float64


Debido a que el porcentaje de nulos no es homogeneo en todas las columnas se infiere que en algunas filas a veces hay solo 1 na, en otras más de 2, etc. Por lo anterior, y con el fin de mantener la mayor cantidad de datos, solo haremos dropna a las filas donde todos sus valores sean NA.



In [44]:
# especificando "all" dropea Nas solo allí donde toda la fila tiene NA
df_juegos = df_juegos.dropna(how="all").reset_index(drop=True)
df_juegos.shape
#notamos que la base ha disminuido casi al 25%

(32135, 8)

Exploramos cómo han cambiado las cantidades de nulos


In [45]:
porcentaje_nulos = df_juegos.isna().mean() * 100
print(porcentaje_nulos)

publisher       25.056792
genres          10.216275
app_name         0.006224
release_date     6.432239
price            4.285047
early_access     0.000000
id               0.006224
developer       10.266065
dtype: float64


## **4. Exploramos el resumen de tipo de datos por columna**

Vamos a ver un resumen del tipo de datos que contiene cada columna. Notamos que varias tienen listas en las filas, Pensaremos si después debemos desagregar estos valores. En este momento es claro que debemos hacerlo para el género, de manera que, en los casos donde un registro tiene una lista de géneros, habrá que hacer que cada registro se guarde con cada uno de los géneros por separado. Después trataremos cada columna, allí haremos dropeo de NAs de manera más específica

In [46]:
resumen_tipos_columnas = df_juegos.apply(lambda x: {"tipo_dato": type(x.iloc[0]), "primeros_valores": x.iloc[:5].tolist()})

# Mostrar el resumen
for columna, resumen in resumen_tipos_columnas.items():
    print(f"Columna: {columna}")
    print(f"Tipo de dato: {resumen['tipo_dato']}")
    print(f"Primeros valores: {resumen['primeros_valores']}")
    print("\n")


Columna: publisher
Tipo de dato: <class 'str'>
Primeros valores: ['Kotoshiro', 'Making Fun, Inc.', 'Poolians.com', '彼岸领域', nan]


Columna: genres
Tipo de dato: <class 'list'>
Primeros valores: [['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'], ['Free to Play', 'Indie', 'RPG', 'Strategy'], ['Casual', 'Free to Play', 'Indie', 'Simulation', 'Sports'], ['Action', 'Adventure', 'Casual'], nan]


Columna: app_name
Tipo de dato: <class 'str'>
Primeros valores: ['Lost Summoner Kitty', 'Ironbound', 'Real Pool 3D - Poolians', '弹炸人2222', 'Log Challenge']


Columna: release_date
Tipo de dato: <class 'str'>
Primeros valores: ['2018-01-04', '2018-01-04', '2017-07-24', '2017-12-07', nan]


Columna: price
Tipo de dato: <class 'float'>
Primeros valores: [4.99, 'Free To Play', 'Free to Play', 0.99, 2.99]


Columna: early_access
Tipo de dato: <class 'bool'>
Primeros valores: [False, False, False, False, False]


Columna: id
Tipo de dato: <class 'str'>
Primeros valores: ['761140', '643980', '670290'

**Exploremos los valores unicos en las columnas para darnos una idea de cómo hacer después la limpieza**

In [47]:
for columna in df_juegos.columns:
    if df_juegos[columna].dtype == "O":  
        valores_unicos = df_juegos[columna].explode().unique()
    else:
        valores_unicos = df_juegos[columna].unique()
    print(f"Valores únicos en la columna '{columna}':\n{valores_unicos}\n")


Valores únicos en la columna 'publisher':
['Kotoshiro' 'Making Fun, Inc.' 'Poolians.com' ... 'OrtiGames/OrtiSoft'
 'INGAME' 'Bidoniera Games']

Valores únicos en la columna 'genres':
['Action' 'Casual' 'Indie' 'Simulation' 'Strategy' 'Free to Play' 'RPG'
 'Sports' 'Adventure' nan 'Racing' 'Early Access' 'Massively Multiplayer'
 'Animation &amp; Modeling' 'Video Production' 'Utilities'
 'Web Publishing' 'Education' 'Software Training'
 'Design &amp; Illustration' 'Audio Production' 'Photo Editing'
 'Accounting']

Valores únicos en la columna 'app_name':
['Lost Summoner Kitty' 'Ironbound' 'Real Pool 3D - Poolians' ...
 'Russian Roads' 'EXIT 2 - Directions' 'Maze Run VR']

Valores únicos en la columna 'release_date':
['2018-01-04' '2017-07-24' '2017-12-07' ... '2016-11-19' 'January 2018'
 '2018-10-01']

Valores únicos en la columna 'price':
[4.99 'Free To Play' 'Free to Play' 0.99 2.99 3.99 9.99 18.99 29.99 nan
 'Free' 10.99 1.59 14.99 1.99 59.99 8.99 6.99 7.99 39.99 19.99 7.49 12.99
 5.9

**Vamos a poner la columna id al principio para ayudar a visualizar en lo posterior**

In [48]:
column_order = ["id", "publisher", "genres", "app_name", "release_date", "price", "early_access", "developer"]
df_juegos = df_juegos[column_order]


## **5. Eliminar duplicados**

**Explorar la existencia de duplicados**

In [49]:
# utilizamos el id para explorar la presencia de duplicados en este campo
duplicados_por_id = (lambda df, columna: df[df.duplicated(subset=columna, keep=False)])(df_juegos, "id")

print("filas duplicadas por id")
duplicados_por_id


filas duplicadas por id


,id,publisher,genres,app_name,release_date,price,early_access,developer
74,NaN,NaN,NaN,NaN,NaN,19.99,False,NaN
13894,612880,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,2017-10-26,59.99,False,Machine Games
14573,612880,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,2017-10-26,59.99,False,Machine Games
30961,NaN,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,2012-09-07,19.99,False,"Rocksteady Studios,Feral Interactive (Mac)"


**procedemos a eliminar el registro 74, que es el que no tiene casi datos y cualquiera de los dos registros duplicados que no son NaN**

In [50]:
df_juegos = df_juegos.drop([74, 14573])


**Comprobamos que ya no hay registros duplicados**

In [51]:
duplicados_por_id = (lambda df, columna: df[df.duplicated(subset=columna, keep=False)])(df_juegos, "id")

# Mostrar resultados
print("filas duplicadas por id")
duplicados_por_id

filas duplicadas por id


,id,publisher,genres,app_name,release_date,price,early_access,developer


## **6. Tratamiento de la columna  "release_date"**

Crearemos una columna conviertiendo los valores a fechas. Después veremos los valores que no pudieron ser convertidos e intentaremos obtener igual el año si es posible para dejarlo en esa columna

In [52]:
# a partir de release_date creamos una nueva columna con formato de fecha llamada "fecha convertida"
df_juegos["fecha_convertida"] = pd.to_datetime(df_juegos["release_date"], errors="coerce", format="%Y-%m-%d")

# Obtener los valores que no se pudieron convertir
valores_no_convertidos = df_juegos[df_juegos["fecha_convertida"].isna()]["release_date"].unique()

# Mostrar los valores no convertidos
print("Valores no convertidos a 'aaaa-mm-dd':")
print(valores_no_convertidos)


Valores no convertidos a 'aaaa-mm-dd':
[nan 'Soon..' '2017' 'Beta测试已开启' 'Jun 2009' 'Oct 2010' 'Feb 2011'
 'Aug 2014' 'Sep 2014' 'Apr 2015' 'Apr 2016' 'Jul 2016' 'June 2016'
 'Coming Soon' 'Q2 2017' 'TBA' "When it's done" 'coming soon' 'Q2 2018'
 '2018' 'Winter 2017' 'soon' 'Mar 2018' '14 July' 'Jul 2017' 'Summer 2017'
 'Spring 2018' 'Winter 2018' 'To be Announced' 'October 2017' 'TBD'
 'Fall 2017' 'Nov 2017' 'Q1 2018' 'Dec 2017' 'Январь 2018' 'Soon'
 'First quarter of 2018' 'H2 2018' '21 Jun, 2017' '2018年初頭発売予定'
 'Please wait warmly' 'early access' 'SOON' 'Feb 2018'
 'Coming Soon/Próximamente' 'August 2017' '2018 [Now get free Pre Alpha]'
 '1st Quarter 2018' 'Jan 2018' '预热群52756441' '2018年1月' 'Coming soon'
 'Fall 2018'
 '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵'
 '"""Soon"""' 'Spring 2017' 'Demo coming soon.' 'Coming 2017'
 'Not yet available' "C'est bientôt...                    (ou pas)"
 'January 2019' 'Q4 2017' 'Early 2018'
 'Datach

Podemos notar que hay varios valores que tienen datos de fecha que no fueron convertidos en automático. Intentaremoss extraer el año de los valores no convertidos, como sólo nos interesa el año para analisis posteriores a estos valores les asumiremos como mm y dd 01-01 respectivamente.

In [53]:
# Función para extraer año y formatear como "aaaa-mm-dd"
def extraer_anio_y_formatear(valor):
    match = re.search(r'\b(?:19|20)\d{2}\b', str(valor))
    if match:
        anio = match.group()
        return f"{anio}-01-01"
    else:
        return None

# aplicamos la función para extraer año y formatear a la columna "release_date"
df_juegos["fecha_convertida"] = df_juegos["release_date"].apply(extraer_anio_y_formatear)


Comprobamos los valores unicos para "fecha_convertida"

In [54]:
valores_unicos = df_juegos["fecha_convertida"].unique()
print(valores_unicos)

['2018-01-01' '2017-01-01' None '1997-01-01' '1998-01-01' '2016-01-01'
 '2006-01-01' '2005-01-01' '2003-01-01' '2007-01-01' '2002-01-01'
 '2000-01-01' '1995-01-01' '1996-01-01' '1994-01-01' '2001-01-01'
 '1993-01-01' '2004-01-01' '1999-01-01' '2008-01-01' '2009-01-01'
 '1992-01-01' '1989-01-01' '2010-01-01' '2011-01-01' '2013-01-01'
 '2012-01-01' '2014-01-01' '1983-01-01' '1984-01-01' '2015-01-01'
 '1990-01-01' '1988-01-01' '1991-01-01' '1985-01-01' '1982-01-01'
 '1987-01-01' '1981-01-01' '1986-01-01' '2021-01-01' '2019-01-01'
 '1975-01-01' '1970-01-01' '1980-01-01']


Una vez hecho lo anterior podemos convertir lo que no tiene formato de fecha a valores perdidos y extraer el año para crear "annio lanzamiento". Eliminamos después la columna "release_date" ya que no nos va a ser útil después.

In [55]:
# convertimos la columna "fecha_convertida" a datetime con errores o nones convertidos a NaN
df_juegos["fecha_convertida"] = pd.to_datetime(df_juegos["fecha_convertida"], errors="coerce")

# creamos la columna "annio_lanzamiento" con el año extraído unicamente de "fecha convertida"
df_juegos["annio_lanzamiento"] = df_juegos["fecha_convertida"].dt.year

# eliminamos las columnas "release_date" y "fecha_convertida"
df_juegos = df_juegos.drop(["release_date", "fecha_convertida"], axis=1)
df_juegos


,id,publisher,genres,app_name,price,early_access,developer,annio_lanzamiento
0,761140,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,False,Kotoshiro,2018.0
1,643980,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Free To Play,False,Secret Level SRL,2018.0
2,670290,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Free to Play,False,Poolians.com,2017.0
3,767400,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,0.99,False,彼岸领域,2017.0
4,773570,NaN,NaN,Log Challenge,2.99,False,NaN,NaN
...,...,...,...,...,...,...,...,...
32130,773640,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,1.99,False,"Nikita ""Ghost_RUS""",2018.0
32131,733530,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,4.99,False,Sacada,2018.0
32132,610660,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,1.99,False,Laush Dmitriy Sergeevich,2018.0
32133,658870,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,4.99,False,"xropi,stev3ns",2017.0


observamos los valores unicos de annio_lanzamiento

In [56]:
valores_unicos_year = df_juegos["annio_lanzamiento"].unique()
print(valores_unicos_year)


[2018. 2017.   nan 1997. 1998. 2016. 2006. 2005. 2003. 2007. 2002. 2000.
 1995. 1996. 1994. 2001. 1993. 2004. 1999. 2008. 2009. 1992. 1989. 2010.
 2011. 2013. 2012. 2014. 1983. 1984. 2015. 1990. 1988. 1991. 1985. 1982.
 1987. 1981. 1986. 2021. 2019. 1975. 1970. 1980.]


convertimos los valores de annio_lanzamiento de float a INT

In [57]:
df_juegos["annio_lanzamiento"] = pd.to_numeric(df_juegos["annio_lanzamiento"], errors="coerce").astype("Int64")

df_juegos

,id,publisher,genres,app_name,price,early_access,developer,annio_lanzamiento
0,761140,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,False,Kotoshiro,2018
1,643980,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Free To Play,False,Secret Level SRL,2018
2,670290,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Free to Play,False,Poolians.com,2017
3,767400,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,0.99,False,彼岸领域,2017
4,773570,NaN,NaN,Log Challenge,2.99,False,NaN,<NA>
...,...,...,...,...,...,...,...,...
32130,773640,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,1.99,False,"Nikita ""Ghost_RUS""",2018
32131,733530,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,4.99,False,Sacada,2018
32132,610660,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,1.99,False,Laush Dmitriy Sergeevich,2018
32133,658870,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,4.99,False,"xropi,stev3ns",2017


observamos que lo que no pudo coercionar se conviertio a NA con una nomenclatura específica por el valor de la columna. Luego veremos cómo tratar esto

vamos a observar ahora los valores unicos para annio_lanzamiento. No debe haber valores que no sean años y NAs

In [58]:
df_juegos["annio_lanzamiento"].unique()

<IntegerArray>
[2018, 2017, <NA>, 1997, 1998, 2016, 2006, 2005, 2003, 2007, 2002, 2000, 1995,
 1996, 1994, 2001, 1993, 2004, 1999, 2008, 2009, 1992, 1989, 2010, 2011, 2013,
 2012, 2014, 1983, 1984, 2015, 1990, 1988, 1991, 1985, 1982, 1987, 1981, 1986,
 2021, 2019, 1975, 1970, 1980]
Length: 44, dtype: Int64

## **7. Tratamiento de la columna "price"**

Vamos a explorar los valores unicos de precio para ver qué tipo de datos hay. Para luego quitar las cadenas (que no son precios) y  cambiarlas por 0.

In [59]:
df_juegos["price"].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, nan, 'Free', 10.99, 1.59, 14.99, 1.99, 59.99, 8.99,
       6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25,
       24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99,
       179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99,
       22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95,
       499.99, 16.06, 4.68, 131

convertimos los valores no numericos a Na y luego los cambiamos a 0, asumiendo que la info que dan en la cadena puede implicar que es un juego gratuito. Lo anterior excenta a los valores que podemos ver que no serían free games

In [60]:
#convertimos los valores que podemos identificar como precios de partida
df_juegos["price"] = df_juegos["price"].replace("Starting at $499.00", 499.00)
df_juegos["price"] = df_juegos["price"].replace("Starting at $449.00", 449.00)

# convertimos la columna "price" a tipo de dato numérico y manejamos los valores
#no numéricos como 0
df_juegos["price"] = pd.to_numeric(df_juegos["price"], errors="coerce").fillna(0)

# visualizamos df_juegos
df_juegos


,id,publisher,genres,app_name,price,early_access,developer,annio_lanzamiento
0,761140,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,False,Kotoshiro,2018
1,643980,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,0.00,False,Secret Level SRL,2018
2,670290,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,0.00,False,Poolians.com,2017
3,767400,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,0.99,False,彼岸领域,2017
4,773570,NaN,NaN,Log Challenge,2.99,False,NaN,<NA>
...,...,...,...,...,...,...,...,...
32130,773640,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,1.99,False,"Nikita ""Ghost_RUS""",2018
32131,733530,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,4.99,False,Sacada,2018
32132,610660,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,1.99,False,Laush Dmitriy Sergeevich,2018
32133,658870,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,4.99,False,"xropi,stev3ns",2017


Exploramos de nuevo los valores unicos de price para ver si necesitamos más cambios

In [61]:
df_juegos["price"].unique()

array([4.9900e+00, 0.0000e+00, 9.9000e-01, 2.9900e+00, 3.9900e+00,
       9.9900e+00, 1.8990e+01, 2.9990e+01, 1.0990e+01, 1.5900e+00,
       1.4990e+01, 1.9900e+00, 5.9990e+01, 8.9900e+00, 6.9900e+00,
       7.9900e+00, 3.9990e+01, 1.9990e+01, 7.4900e+00, 1.2990e+01,
       5.9900e+00, 2.4900e+00, 1.5990e+01, 1.2500e+00, 2.4990e+01,
       1.7990e+01, 6.1990e+01, 3.4900e+00, 1.1990e+01, 1.3990e+01,
       3.4990e+01, 7.4760e+01, 1.4900e+00, 3.2990e+01, 9.9990e+01,
       1.4950e+01, 6.9990e+01, 1.6990e+01, 7.9990e+01, 4.9990e+01,
       5.0000e+00, 4.4990e+01, 1.3980e+01, 2.9960e+01, 1.1999e+02,
       1.0999e+02, 1.4999e+02, 7.7171e+02, 2.1990e+01, 8.9990e+01,
       9.8000e-01, 1.3992e+02, 4.2900e+00, 6.4990e+01, 5.4990e+01,
       7.4990e+01, 8.9000e-01, 5.0000e-01, 2.9999e+02, 1.2900e+00,
       3.0000e+00, 1.5000e+01, 5.4900e+00, 2.3990e+01, 4.9000e+01,
       2.0990e+01, 1.0930e+01, 1.3900e+00, 3.6990e+01, 4.4900e+00,
       2.0000e+00, 4.0000e+00, 9.0000e+00, 2.3499e+02, 1.9500e

No parece haber ya valores faltantes

## **8 Poner etiqueta "sin dato" a columnas string con NAs**

Por otro lado, como probablemente después necesitemos que las columnas con NA tengan "una etiqueta"(para que sirvan a algunos analises del EDA) vamos a llenar esas columnas con "Sin dato". Recordemos que, del apartado donde vimos el porcentaje de Nas, sabemos que las columnas que tienen este tipo de valor (y que además son string) son: id, publisher, genres, app_name, tags, reviews_url, developer.

In [62]:
# Lista de columnas a procesar
columnas_a_procesar = ["id", "publisher", "genres", "app_name", "developer", "annio_lanzamiento"]
# Convertir "annio_lanzamiento" a string
df_juegos["annio_lanzamiento"] = df_juegos["annio_lanzamiento"].astype(str)

# Reemplazar NaN, <NA>, y None con "Sin dato" en las columnas especificadas
reemplazos = {
    np.nan: "Sin dato",
    pd.NA: "Sin dato",
    None: "Sin dato",
}
df_juegos[columnas_a_procesar] = df_juegos[columnas_a_procesar].replace(reemplazos)

# Calcular el porcentaje de valores nulos
porcentaje_nulos = df_juegos.isna().mean() * 100
print(porcentaje_nulos)


id                   0.0
publisher            0.0
genres               0.0
app_name             0.0
price                0.0
early_access         0.0
developer            0.0
annio_lanzamiento    0.0
dtype: float64


## **9. Tratamiento de la columna "genres"**
**Desagregar el género de las filas donde está como una lista**

Ahora, como vimos hace rato, la columna género tiene en algunas filas una lista, hay que hacer que cada elemento de esa lista se asocie con un id y la información especifica, es decir, hay que desagruparla. Por lo anterior, se van a duplicar los ids para diferentes generos.

In [63]:
# "desempaquetamos" la columna
df_juegos = df_juegos.explode("genres")

#tratamos sus respectivos valores perdidos
df_juegos = df_juegos.dropna(subset=["genres"])

## visualizamos cómo quedó la columna
df_juegos

,id,publisher,genres,app_name,price,early_access,developer,annio_lanzamiento
0,761140,Kotoshiro,Action,Lost Summoner Kitty,4.99,False,Kotoshiro,2018
0,761140,Kotoshiro,Casual,Lost Summoner Kitty,4.99,False,Kotoshiro,2018
0,761140,Kotoshiro,Indie,Lost Summoner Kitty,4.99,False,Kotoshiro,2018
0,761140,Kotoshiro,Simulation,Lost Summoner Kitty,4.99,False,Kotoshiro,2018
0,761140,Kotoshiro,Strategy,Lost Summoner Kitty,4.99,False,Kotoshiro,2018
...,...,...,...,...,...,...,...,...
32132,610660,Laush Studio,Racing,Russian Roads,1.99,False,Laush Dmitriy Sergeevich,2018
32132,610660,Laush Studio,Simulation,Russian Roads,1.99,False,Laush Dmitriy Sergeevich,2018
32133,658870,SIXNAILS,Casual,EXIT 2 - Directions,4.99,False,"xropi,stev3ns",2017
32133,658870,SIXNAILS,Indie,EXIT 2 - Directions,4.99,False,"xropi,stev3ns",2017


Vamos a ver ahora los valores unicos para genre. Por si allí se nos hubiera colado algún dato Na o de otro tipo. Podemos observar que no y que las categorías parecen ser correctas

In [64]:
df_juegos["genres"].unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Sin dato', 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

## **10. Tratamiento de la columna developer**

veamos la frecuencia de los valores unicos para la columna developer

In [65]:
valores_uniq_develop = df_juegos['developer'].value_counts().head(20)
print(f"Desarrolladores únicos: {valores_uniq_develop}")

Desarrolladores únicos: developer
Sin dato                      3478
Ubisoft - San Francisco       2516
SmiteWorks USA, LLC           2364
DL Softworks                   495
Ronimo Games                   379
Paradox Development Studio     311
Musopia                        285
Arcane Raise                   266
Dovetail Games                 256
Choice of Games                248
Boogygames Studios             236
KOEI TECMO GAMES CO., LTD.     233
Milestone S.r.l.               211
Magic Pixel Kft.               207
Capcom                         176
Warfare Studios                160
Aldorlea Games                 150
GameUS Inc.                    150
SCS Software                   144
Llama Software                 141
Name: count, dtype: int64


Es probable que la columna developer no se encuentre estandarizada en todas sus filas. Vamos a convertir los nombres a minuscula, a eliminar espacios que no se necesiten 

In [66]:
# si el nombre es una cadena procede a la normalización
df_juegos["developer"] = df_juegos["developer"].apply(lambda name: re.sub(r"\s+", " ", re.sub(r"[^a-zA-Z\s]", "", name.lower().strip())) if isinstance(name, str) else name)


## **11. Visualización final de los tipos de dato por columna**



Veamos qué tipo de dato contienen las columnas hasta el momento

In [67]:
resumen_columnas = df_juegos.apply(lambda x: {"tipo_dato": type(x.iloc[0]), "primeros_valores": x.iloc[:5].tolist()})

# Mostrar el resumen
for columna, resumen in resumen_columnas.items():
    print(f"Columna: {columna}")
    print(f"Tipo de dato: {resumen['tipo_dato']}")
    print(f"Primeros valores: {resumen['primeros_valores']}")
    print("\n")

Columna: id
Tipo de dato: <class 'str'>
Primeros valores: ['761140', '761140', '761140', '761140', '761140']


Columna: publisher
Tipo de dato: <class 'str'>
Primeros valores: ['Kotoshiro', 'Kotoshiro', 'Kotoshiro', 'Kotoshiro', 'Kotoshiro']


Columna: genres
Tipo de dato: <class 'str'>
Primeros valores: ['Action', 'Casual', 'Indie', 'Simulation', 'Strategy']


Columna: app_name
Tipo de dato: <class 'str'>
Primeros valores: ['Lost Summoner Kitty', 'Lost Summoner Kitty', 'Lost Summoner Kitty', 'Lost Summoner Kitty', 'Lost Summoner Kitty']


Columna: price
Tipo de dato: <class 'numpy.float64'>
Primeros valores: [4.99, 4.99, 4.99, 4.99, 4.99]


Columna: early_access
Tipo de dato: <class 'bool'>
Primeros valores: [False, False, False, False, False]


Columna: developer
Tipo de dato: <class 'str'>
Primeros valores: ['kotoshiro', 'kotoshiro', 'kotoshiro', 'kotoshiro', 'kotoshiro']


Columna: annio_lanzamiento
Tipo de dato: <class 'str'>
Primeros valores: ['2018', '2018', '2018', '2018', '201

In [68]:
df_juegos

,id,publisher,genres,app_name,price,early_access,developer,annio_lanzamiento
0,761140,Kotoshiro,Action,Lost Summoner Kitty,4.99,False,kotoshiro,2018
0,761140,Kotoshiro,Casual,Lost Summoner Kitty,4.99,False,kotoshiro,2018
0,761140,Kotoshiro,Indie,Lost Summoner Kitty,4.99,False,kotoshiro,2018
0,761140,Kotoshiro,Simulation,Lost Summoner Kitty,4.99,False,kotoshiro,2018
0,761140,Kotoshiro,Strategy,Lost Summoner Kitty,4.99,False,kotoshiro,2018
...,...,...,...,...,...,...,...,...
32132,610660,Laush Studio,Racing,Russian Roads,1.99,False,laush dmitriy sergeevich,2018
32132,610660,Laush Studio,Simulation,Russian Roads,1.99,False,laush dmitriy sergeevich,2018
32133,658870,SIXNAILS,Casual,EXIT 2 - Directions,4.99,False,xropistevns,2017
32133,658870,SIXNAILS,Indie,EXIT 2 - Directions,4.99,False,xropistevns,2017


renombrar id a item_id y annio_lanzamiento a year, como en el ipynb etl 2




In [69]:
df_juegos.rename(columns={"id": "item_id", "annio_lanzamiento": "year"}, inplace=True)


In [70]:
df_juegos

,item_id,publisher,genres,app_name,price,early_access,developer,year
0,761140,Kotoshiro,Action,Lost Summoner Kitty,4.99,False,kotoshiro,2018
0,761140,Kotoshiro,Casual,Lost Summoner Kitty,4.99,False,kotoshiro,2018
0,761140,Kotoshiro,Indie,Lost Summoner Kitty,4.99,False,kotoshiro,2018
0,761140,Kotoshiro,Simulation,Lost Summoner Kitty,4.99,False,kotoshiro,2018
0,761140,Kotoshiro,Strategy,Lost Summoner Kitty,4.99,False,kotoshiro,2018
...,...,...,...,...,...,...,...,...
32132,610660,Laush Studio,Racing,Russian Roads,1.99,False,laush dmitriy sergeevich,2018
32132,610660,Laush Studio,Simulation,Russian Roads,1.99,False,laush dmitriy sergeevich,2018
32133,658870,SIXNAILS,Casual,EXIT 2 - Directions,4.99,False,xropistevns,2017
32133,658870,SIXNAILS,Indie,EXIT 2 - Directions,4.99,False,xropistevns,2017


## **12. últimos ajustes**

busqueda de NAs pendientes

In [71]:
# Se ve que year tiene na con el formato <NA>, hay que quitarlos con una función, vamos a ver si hay en otras
# columnas otras formas de NA que se hayan pasado desapercibidas
# Establecemos las presentaciones posibles de valores nulos como cadenas
representaciones_nulos = ["<NA>", "None", "NA", "<na>", "none", "na", "N/A", "n.a.", "n.a", "N/A/", "missing", "null", ""]

# creamos una máscara que identifique las filas con alguna representación de valor nulo que pusimos arriba
mask_nulos = df_juegos.apply(lambda col: col.astype(str).isin(representaciones_nulos)).any(axis=1)

# Filtrar y mostrar las filas que contienen alguna representación de valor nulo
filas_nulos_cadena = df_juegos[mask_nulos]
filas_nulos_cadena


,item_id,publisher,genres,app_name,price,early_access,developer,year
3,767400,彼岸领域,Action,弹炸人2222,0.99,False,,2017
3,767400,彼岸领域,Adventure,弹炸人2222,0.99,False,,2017
3,767400,彼岸领域,Casual,弹炸人2222,0.99,False,,2017
4,773570,Sin dato,Sin dato,Log Challenge,2.99,False,sin dato,<NA>
10,768570,Qucheza,Action,Uncanny Islands,0.00,True,qucheza,<NA>
...,...,...,...,...,...,...,...,...
32100,754350,杭州分浪网络科技有限公司,Free to Play,忍者村大战2,0.00,False,,2018
32100,754350,杭州分浪网络科技有限公司,Massively Multiplayer,忍者村大战2,0.00,False,,2018
32100,754350,杭州分浪网络科技有限公司,RPG,忍者村大战2,0.00,False,,2018
32100,754350,杭州分浪网络科技有限公司,Strategy,忍者村大战2,0.00,False,,2018


In [72]:
# Representaciones posibles de valores nulos como cadenas, tambien de valores vacios. 
# Esto ultimo porque estandarizamos developer después de cambiar a "Sin dato"
representaciones_nulos = ["<NA>", "None", "NA", "<na>", "none", "na", "N/A", "n.a.", "n.a", "N/A/", "missing", "null", "sin dato", ""]

# Crear una máscara que identifique las filas con alguna representación de valor nulo
mask_nulos = df_juegos.apply(lambda col: col.astype(str).isin(representaciones_nulos)).any(axis=1)

# Filtrar y mostrar las filas que contienen alguna representación de valor nulo
filas_nulos_cadena = df_juegos[mask_nulos]
filas_nulos_cadena

,item_id,publisher,genres,app_name,price,early_access,developer,year
3,767400,彼岸领域,Action,弹炸人2222,0.99,False,,2017
3,767400,彼岸领域,Adventure,弹炸人2222,0.99,False,,2017
3,767400,彼岸领域,Casual,弹炸人2222,0.99,False,,2017
4,773570,Sin dato,Sin dato,Log Challenge,2.99,False,sin dato,<NA>
10,768570,Qucheza,Action,Uncanny Islands,0.00,True,qucheza,<NA>
...,...,...,...,...,...,...,...,...
32118,775640,Sin dato,Sin dato,Robotpencil Presents: Exercise: Brushwork,3.99,False,sin dato,2018
32119,777930,Sin dato,Sin dato,Robotpencil Presents: Creative Composition,3.99,False,sin dato,2018
32120,775370,Sin dato,Sin dato,The Gamble House,4.99,False,sin dato,2016
32121,777950,Sin dato,Sin dato,Kalen Chock Presents: 2017 Free Tutorial,0.00,False,sin dato,2018


eliminamos esos valores

In [73]:
representaciones_nulos = ["<NA>", "None", "NA", "<na>", "none", "na", "N/A", "n.a.", "n.a", "N/A/", "missing", "null", "sin dato", ""]

# Reemplazar todas las representaciones de valores nulos por "Sin dato"
df_juegos.replace(representaciones_nulos, "Sin dato", inplace=True)


comprobamos que se han eliminado

In [74]:
# Representaciones posibles de valores nulos como cadenas, tambien de valores vacios. 
# Esto ultimo porque estandarizamos developer después de cambiar a "Sin dato"
representaciones_nulos = ["<NA>", "None", "NA", "<na>", "none", "na", "N/A", "n.a.", "n.a", "N/A/", "missing", "null", "sin dato", ""]

# Crear una máscara que identifique las filas con alguna representación de valor nulo
mask_nulos = df_juegos.apply(lambda col: col.astype(str).isin(representaciones_nulos)).any(axis=1)

# Filtrar y mostrar las filas que contienen alguna representación de valor nulo
filas_nulos_cadena = df_juegos[mask_nulos]
filas_nulos_cadena

,item_id,publisher,genres,app_name,price,early_access,developer,year


comprobamos que a nivel de df no se detectan valores nulos

In [75]:
df_juegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74835 entries, 0 to 32134
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       74835 non-null  object 
 1   publisher     74835 non-null  object 
 2   genres        74835 non-null  object 
 3   app_name      74835 non-null  object 
 4   price         74835 non-null  float64
 5   early_access  74835 non-null  bool   
 6   developer     74835 non-null  object 
 7   year          74835 non-null  object 
dtypes: bool(1), float64(1), object(6)
memory usage: 4.6+ MB


FINALMENTE YA NO HAY NADA DE VALORES NULOS DE LOS QUE SE UBICAN COMO CADENAS, NI DE LOS QUE SE UBICAN POR LA FUNCION

## **13. Exportar DF curado**

Ya revisado lo anterior el df está limpio. Se exporta ahora. Para ahorrar espacio lo guardamos en parquet


In [76]:
df_juegos.to_parquet("bases/steam_games_curado.parquet")

c:\Users\Legion 5 3060\.conda\envs\SoyHenryProyectconda\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


## **14. Importar df curado**

In [77]:
ruta_archivo_parquet = "bases/steam_games_curado.parquet"

# Importar el archivo Parquet como un DataFrame
df_parquet = pd.read_parquet(ruta_archivo_parquet)

probamos que funciona bien la importacion. No deben reaparecer valores nulos

In [78]:
print(df_parquet.info())

<class 'pandas.core.frame.DataFrame'>
Index: 74835 entries, 0 to 32134
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       74835 non-null  object 
 1   publisher     74835 non-null  object 
 2   genres        74835 non-null  object 
 3   app_name      74835 non-null  object 
 4   price         74835 non-null  float64
 5   early_access  74835 non-null  bool   
 6   developer     74835 non-null  object 
 7   year          74835 non-null  object 
dtypes: bool(1), float64(1), object(6)
memory usage: 4.6+ MB
None


Importa correctamente, no aparecen valores nulos tal y como fue guardada la base.  